<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/BGL2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#upload log files
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/X_train_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/Xabnorm_test_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/Xnorm_test_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/log2index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/reduced_index2embed'

--2023-09-05 13:32:36--  https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/X_train_index
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9341136 (8.9M) [text/plain]
Saving to: ‘X_train_index’

X_train_index       100%[===================>]   8.91M  --.-KB/s    in 0.05s   

2023-09-05 13:32:37 (174 MB/s) - ‘X_train_index’ saved [9341136/9341136]

--2023-09-05 13:32:37--  https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/Xabnorm_test_index
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response...

In [2]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np
from tqdm import tqdm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [25]:

with open('/content/log2index') as f:
    templates = [line.strip() for line in f.readlines()]

In [27]:
templates[:5]

['ras kernel info instruction cache parity error corrected',
 'ras linkcard info midplaneswitchcontroller performing bit sparing on bit',
 'ras kernel info generating',
 'ras kernel info ddr errors s detected and corrected on rank symbol bit',
 'ras kernel info edram error s dcr detected and corrected']

In [3]:
#Json file name
with open('/content/reduced_index2embed') as f:
    embeddings = [json.loads(line) for line in f.readlines()]

len(embeddings[0])

60

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
embed = np.array(embeddings)
matrix = np.zeros((len(templates),len(templates)))
max_similarity = 0

for i in range(len(templates)):
  for j in range(i+1,len(templates)) :

      similarity = cosine_similarity(embed[i].reshape(1, -1),embed[j].reshape(1, -1))
      matrix[i,j] = similarity

      if (similarity > 0.85) & (similarity < 0.90) :
        print('similarity = ',similarity)
        print(templates[i])
        print(templates[j])
        print('-----------------------------------')

similarity =  [[0.86446773]]
ras kernel info instruction cache parity error corrected
ras kernel info data cache flush parity error detected attempting to correct
-----------------------------------
similarity =  [[0.89088115]]
ras kernel info ddr activating redundant bit steering rank symbol
ras kernel info ddr unable to steer rank symbol rank is already steering symbol due to multiple symbols being over the correctable
-----------------------------------
similarity =  [[0.87308842]]
ras kernel info ddr activating redundant bit steering rank symbol
ras kernel info ddr unable to steer rank symbol rank is already steering symbol due to multiple symbols being over the correctable e
-----------------------------------
similarity =  [[0.89107637]]
ras kernel info ddr activating redundant bit steering rank symbol
ras kernel info ddr unable to steer rank symbol rank is already steering symbol due to multiple symbols being over the correctabl
-----------------------------------
similarity =  

KeyboardInterrupt: ignored

In [35]:
# Define the LSTM Autoencoder model with dropout
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, sequence_length, dropout_prob): #input_dim= embeddings_dim
        super(LSTMAutoencoder, self).__init__()
        self.sequence_length = sequence_length
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True, dropout=dropout_prob)
        self.decoder = nn.LSTM(hidden_dim * 2, input_dim, num_layers, batch_first=True, dropout=dropout_prob)
        #self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, x):
        encoded, _ = self.encoder(x)
        #print(encoded.shape)
        encoded = encoded[:,-1:,:] # output of last cell
        #encoded = self.fc(encoded)
        #print(encoded.shape)
        input_decode = torch.tile(encoded, (1, self.sequence_length, 1))
        decoded, _ = self.decoder(input_decode)
        return decoded

In [36]:
sequence_length = 5
input_size = len(embeddings[0]) #embedding vector dimension
hidden_dim = 64
num_layers = 2
dropout_prob = 0.2

model = LSTMAutoencoder(input_size, hidden_dim, 2, sequence_length, dropout_prob).to(device)
model

LSTMAutoencoder(
  (encoder): LSTM(60, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (decoder): LSTM(128, 60, num_layers=2, batch_first=True, dropout=0.2)
)

In [37]:
data = torch.randn(1,sequence_length, input_size) #batch_size=1
model(data.to(device)).shape

torch.Size([1, 5, 60])

In [38]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 238720


In [39]:
window_size = sequence_length
num_sessions = 0
sequences = {}

line = [i for i in range(20)]
print(line)

for i in range(len(line) - window_size):
    seq = tuple(line[i:i + window_size])
    if seq not in sequences:
      sequences[seq] = [ embeddings[i] for i in seq]


print(sequences.keys())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
dict_keys([(0, 1, 2, 3, 4), (1, 2, 3, 4, 5), (2, 3, 4, 5, 6), (3, 4, 5, 6, 7), (4, 5, 6, 7, 8), (5, 6, 7, 8, 9), (6, 7, 8, 9, 10), (7, 8, 9, 10, 11), (8, 9, 10, 11, 12), (9, 10, 11, 12, 13), (10, 11, 12, 13, 14), (11, 12, 13, 14, 15), (12, 13, 14, 15, 16), (13, 14, 15, 16, 17), (14, 15, 16, 17, 18)])


In [40]:
name = 'X_train_index'
window_size = sequence_length
sequences = {}
#outputs = []
with open('/content/' + name, 'r') as f:
        for row in f:
            line = [ int(i) for i in row.strip().split()]

            for i in range(len(line) - window_size):
                seq = tuple(line[i:i + window_size])
                if seq not in sequences:
                  sequences[seq] = [ embeddings[i] for i in seq]

print('the length of dictionary : ', len(sequences))

dataset = TensorDataset(torch.tensor(list(sequences.values()), dtype=torch.float))

the length of dictionary :  24079


In [41]:
list(sequences.keys())

[(0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 2),
 (0, 0, 0, 2, 2),
 (0, 0, 2, 2, 0),
 (0, 2, 2, 0, 2),
 (2, 2, 0, 2, 2),
 (0, 2, 2, 2, 0),
 (2, 2, 2, 0, 2),
 (2, 0, 2, 2, 2),
 (2, 2, 2, 0, 0),
 (2, 2, 0, 0, 2),
 (2, 0, 0, 2, 2),
 (0, 0, 2, 2, 2),
 (0, 2, 2, 2, 2),
 (2, 2, 2, 2, 0),
 (2, 2, 0, 2, 0),
 (2, 0, 2, 0, 0),
 (0, 2, 0, 0, 0),
 (0, 3, 4, 5, 6),
 (3, 4, 5, 6, 4),
 (4, 5, 6, 4, 4),
 (5, 6, 4, 4, 2),
 (6, 4, 4, 2, 2),
 (4, 4, 2, 2, 2),
 (4, 2, 2, 2, 2),
 (2, 2, 2, 2, 2),
 (2, 2, 2, 2, 1),
 (2, 2, 2, 1, 2),
 (2, 2, 1, 2, 2),
 (2, 1, 2, 2, 2),
 (2, 1, 7, 8, 4),
 (1, 7, 8, 4, 3),
 (7, 8, 4, 3, 3),
 (8, 4, 3, 3, 3),
 (4, 3, 3, 3, 5),
 (3, 3, 3, 5, 3),
 (3, 3, 5, 3, 5),
 (3, 5, 3, 5, 3),
 (5, 3, 5, 3, 5),
 (6, 5, 5, 5, 5),
 (5, 5, 5, 5, 5),
 (5, 5, 6, 2, 2),
 (5, 6, 2, 2, 2),
 (6, 2, 2, 2, 2),
 (2, 2, 2, 2, 9),
 (2, 2, 2, 9, 9),
 (2, 2, 9, 9, 9),
 (2, 9, 9, 9, 9),
 (9, 9, 9, 9, 9),
 (9, 9, 9, 9, 3),
 (9, 9, 9, 3, 3),
 (9, 9, 3, 3, 5),
 (9, 3, 3,

In [42]:
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
print(len(dataloader)) # regarding to batch size


for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

189
torch.Size([128, 5, 60])


In [48]:
def adjust_learning_rate(optimizer, epoch, lr_step=(10,20,30,60), lr_decay_ratio=0.5):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 8
    elif epoch in [1, 2, 3]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer

In [49]:
num_epochs = 60
learning_rate = 0.001

# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [50]:
# Train the model
model.train()
start_time = time.time()
total_step = len(dataloader)
for epoch in tqdm(range(num_epochs), desc="Processing Rows"):  # Loop over the dataset multiple times
    optimizer = adjust_learning_rate(optimizer, epoch)
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, (train_loss*100) / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


Processing Rows:   2%|▏         | 1/60 [00:06<06:43,  6.84s/it]

Epoch [1/60], train_loss: 0.1191


Processing Rows:   3%|▎         | 2/60 [00:14<07:03,  7.29s/it]

Epoch [2/60], train_loss: 0.1193


Processing Rows:   5%|▌         | 3/60 [00:22<07:05,  7.46s/it]

...

Epoch [47/60], train_loss: 0.1000


Processing Rows:  80%|████████  | 48/60 [05:49<01:28,  7.37s/it]

Epoch [48/60], train_loss: 0.0995


Processing Rows:  80%|████████  | 48/60 [05:51<01:27,  7.33s/it]


KeyboardInterrupt: ignored

In [ ]:
# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/BGL1_parameters.pth')

# **EVALUATION**


in cpu
```
my_model = model.load_state_dict(torch.load('model_parameters.pth', map_location=torch.device(device)))

# Put the model in evaluation mode if necessary
model.eval()
```

in cuda


```
model.load_state_dict(torch.load('model_parameters.pth'))

# Put the model in evaluation mode if necessary
model.eval()
```



In [ ]:
len(embeddings)

In [19]:
def generate(name):
    window_size = sequence_length
    hdfs = {} #store the unique sequences and their counts.
    length = 0
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) for i in row.strip().split()]
            hdfs[tuple(line)] = hdfs.get(tuple(line), 0) + 1   #If the tuple is not present in the dictionary, hdfs.get(tuple(ln), 0) returns 0, and the code initializes the count to 1.
            length += 1
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs, length

In [20]:
test_normal_loader, test_normal_length = generate('Xnorm_test_index')
test_abnormal_loader, test_abnormal_length = generate('Xabnorm_test_index')

Number of sessions(Xnorm_test_index): 17191
Number of sessions(Xabnorm_test_index): 3595


In [21]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in tqdm(test_normal_loader.keys(), desc="Processing Rows"):
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                FP += test_normal_loader[line] # numbers of that set we have
                break
  with torch.no_grad():
      for line in tqdm(test_abnormal_loader.keys(), desc="Processing Rows"):
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                TP += test_abnormal_loader[line]
                break
  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = test_abnormal_length - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [53]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in tqdm(test_normal_loader.keys(), desc="Processing Rows"):
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                FP += test_normal_loader[line] # numbers of that set we have
                break
  with torch.no_grad():
      for line in tqdm(test_abnormal_loader.keys(), desc="Processing Rows"):
          losses = 0
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)
              losses += loss.cpu().detach().numpy()
          if (losses>threshold):
            TP += test_abnormal_loader[line]

  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = test_abnormal_length - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [ ]:
threshold = [0.005,0.0055,0.006,0.0065]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ',i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.005


Processing Rows: 100%|██████████| 3595/3595 [01:12<00:00, 49.78it/s]


elapsed_time: 415.178s
false positive (FP): 248, false negative (FN): 419, Precision: 98.766%, Recall: 97.932%, F1-measure: 98.347%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0055


Processing Rows: 100%|██████████| 3595/3595 [01:12<00:00, 49.91it/s]


elapsed_time: 414.291s
false positive (FP): 71, false negative (FN): 439, Precision: 99.643%, Recall: 97.833%, F1-measure: 98.730%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.006


Processing Rows: 100%|██████████| 3595/3595 [01:11<00:00, 50.19it/s]


elapsed_time: 414.628s
false positive (FP): 44, false negative (FN): 448, Precision: 99.778%, Recall: 97.789%, F1-measure: 98.774%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0065


Processing Rows:  89%|████████▊ | 15241/17191 [05:10<00:35, 54.62it/s]

In [ ]:
threshold = [0.007,0.0075,0.008,0.00850]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ', i)
  evaluation(i)